In [5]:
import argparse   
import tensorflow as tf  
from scipy import misc

filelist = '/home/zeng/workspace/tensorflow-workspace/singlelist.txt'
image_paths = [l.strip('\n') for l in
    tf.gfile.FastGFile(filelist, 'r').readlines()]

def go_tf(image_paths):
    nrof_samples = len(image_paths)
    images = np.zeros((nrof_samples, IMAGE_SIZE, IMAGE_SIZE, 3))
    for i in range(nrof_samples):
        img = misc.imread(image_paths[i])
        images[i,:,:,:] = img
        
    return images



In [9]:
if __name__ == '__main__':  
    
    frozen_graph_filename = 'mobilenet_v2_075_frozen.pb'
    
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:  
        graph_def = tf.GraphDef()  
        graph_def.ParseFromString(f.read())  
  
    # We load the graph_def in the default graph  
    with tf.Graph().as_default() as graph:  
        new_input = tf.placeholder(
            tf.float32, shape=[None, 256, 256, 3], name="new_input")
        
        tf.import_graph_def(  
            graph_def, 
            input_map={"input:0": new_input},   
            return_elements=None,   
            name="prefix",   
            op_dict=None,   
            producer_op_list=None  
        )  


    #x = graph.get_tensor_by_name('prefix/Placeholder/inputs_placeholder:0')  
    y = graph.get_tensor_by_name('MobilenetV2/Logits/AvgPool_1a/AvgPool:0')  
    
    input_instance = go_tf(filelist)
    
    with tf.Session(graph=graph) as sess:  
        y_out = sess.run(y, feed_dict={  
            "input:0": input_instance
        })  
        print(y_out) # [[ 0.]] Yay!  
    print ("finish")  


KeyError: "The name 'MobilenetV2/Logits/AvgPool_1a/AvgPool:0' refers to a Tensor which does not exist. The operation, 'MobilenetV2/Logits/AvgPool_1a/AvgPool', does not exist in the graph."